In [13]:
import advent

data = advent.get_lines(22)

# The possible instructions are:
# deal into new stack
# cut N
# deal with increment N

instructions = []
for line in data:
    if line.startswith('deal into new stack'):
        instructions.append(('deal', None))
    elif line.startswith('cut'):
        instructions.append(('cut', int(line.split()[-1])))
    elif line.startswith('deal with increment'):
        instructions.append(('increment', int(line.split()[-1])))

def apply(deck, instruction):
    if instruction[0] == 'deal':
        return deck[::-1]
    elif instruction[0] == 'cut':
        return deck[instruction[1]:] + deck[:instruction[1]]
    elif instruction[0] == 'increment':
        new_deck = [0] * len(deck)
        for i, card in enumerate(deck):
            new_deck[(i * instruction[1]) % len(deck)] = card
        return new_deck
    raise ValueError('Unknown instruction')


In [14]:
deck = list(range(10007))
for instruction in instructions:
    deck = apply(deck, instruction)
print(deck.index(2019))

6638


In [16]:
# Part 2
# Idea: we are going to shuffle only card 2020, but in reverse.
# To deeal with the ridiculous amounts of shuffles we will do some cycle detection
deck_size = 119315717514047
num_instructions = 101741582076661

def reverse(instruction, position):
    if instruction[0] == 'deal':
        return deck_size - 1 - position
    elif instruction[0] == 'cut':
        return (position + instruction[1]) % deck_size
    elif instruction[0] == 'increment':
        # AI helped a lot here. I gave this prompt to chatgpt:
        # 'Given coprime N and M, and some number n. I want to find m such that m*M = n mod N'
        # That pointed me to multiplicative inverse, which is apparently implemented in python 3.8+
        # (found on stackoverflow) N=deck_size, M=instruction[1], n=position
        inverse = pow(instruction[1], -1, deck_size)
        return (position * inverse) % deck_size

def apply_reverse_instructions(position):
    for instruction in reversed(instructions):
        position = reverse(instruction, position)
    return position

print(apply_reverse_instructions(2020))

# Now we need to find the position of card 2020 after num_instructions reverse shuffles
positions = {}
new_position, steps = 2020, 0

while (new_position not in positions) and False:
    positions[new_position] = steps
    new_position = apply_reverse_instructions(new_position)
    steps += 1

print(new_position, steps) # cycle detected!
# Unfortunately this was too optimistic,
# given ~100 trillion positions and shuffles, cycle probably wont happen

57481683403608
2020 0


In [ ]:
# Attempt 2
# I got a slight hint from copilot against my will that the transformation
# is a 'linear function', but no more than that
# After mulling it over, I realized that (reverse operations)
# deal(n) = -n - 1
# cut(n, N) = n + N
# increment(n, N) = n * N^-1
# (N^-1 can just be precomputed since we just need like 50 of them)

# example: deal with increment 7, cut 2, deal with increment 9
# would become (reversed): ((n * 9^-1) + 2) * 7^-1 = n * 9^-1 * 7^-1 + 2 * 7^-1
# Which would always be a linear function of n, let's say A*n + B

# To apply that some trillions of times, we can use 'doubling', so we calculate A, B for
# applying it 2 times, 4 times, 8 times, etc. and then combine them to get the final result
# To double: A(A*n + B) + B = A^2 * n + A*B + B, so A' = A^2, B' = A*B + B

# Note in the explanation above I didn't put % deck_size, but we can do that anywhere pretty much


In [30]:
# Let's first calculate the final A, B for one shuffle
A, B = 1, 0
for instruction in reversed(instructions):
    if instruction[0] == 'deal':
        # -n-1 so -(An+B)-1 = -An - B - 1
        A, B = -A, -B - 1
    elif instruction[0] == 'cut':
        # n + N so (An+B) + N = An + B + N
        A, B = A, B + instruction[1]
    elif instruction[0] == 'increment':
        # n * N^-1 so (An+B) * N^-1 = A * N^-1 * n + B * N^-1
        inverse = pow(instruction[1], -1, deck_size)
        A, B = A * inverse, B * inverse
    A, B = A % deck_size, B % deck_size

# Now we need to calculate A, B for 2, 4, 8, 16, ... shuffles
shuffle_coeffs = {1: (A, B)}
coeff = 1
while coeff < deck_size:
    A, B = shuffle_coeffs[coeff]
    A, B = A * A % deck_size, (A * B + B) % deck_size
    shuffle_coeffs[2 * coeff] = (A, B)
    coeff *= 2
#print(shuffle_coeffs)


In [ ]:
# it feels like this should be commutative, but I'm not sure
# i.e. applying 2 shuffles then 4 should be the same as applying 4 then 2
def combine(coeff1, coeff2):
    A1, B1 = coeff1
    A2, B2 = coeff2
    # Copilot suggested this whole thing of course like a bunch of other things but I double checked it :)
    return (A1 * A2) % deck_size, (A1 * B2 + B1) % deck_size

# It does indeed seem to be commutative :D no mathematical proof here though, I assume it's just true
print(combine(shuffle_coeffs[2], shuffle_coeffs[8]))
print(combine(shuffle_coeffs[8], shuffle_coeffs[2]))

(17972100993572, 115832437357225)
(17972100993572, 115832437357225)


In [33]:
import math

instructions_to_apply = num_instructions

final_A, final_B = 1, 0
while instructions_to_apply > 0:
    highest_power = 2**math.floor(math.log2(instructions_to_apply))
    instructions_to_apply -= highest_power
    final_A, final_B = combine((final_A, final_B), shuffle_coeffs[highest_power])

print(((final_A * 2020) + final_B) % deck_size)

77863024474406
